In [6]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from scipy.stats import norm

def mse(x,y,w): #actually returns RMSE
    assert x.dot(w).shape == y.shape
    return np.sqrt(np.mean(np.square(x.dot(w)-y)))

def log_prob_of_order(x,y,coefs,s,order):
    eps = 1e-6
    y_ = y[order]
    log_prob = np.sum(-np.square(x.dot(coefs)-y_)/(2*(s+eps)**2))
    return log_prob

def calc_relative_error(w0, w):
    w = np.array(w); w0 = np.array(w0)
    w = w.flatten()
    w0 = w0.flatten()
    return np.linalg.norm(w-w0)/np.linalg.norm(w0)

def calc_error(w0, w, multi=False):
    w = np.array(w); w0 = np.array(w0)
    if (multi):
        w = w.reshape(w0.shape)
        return np.linalg.norm(w-w0,axis=1)
    else:
        w = w.flatten()
        w0 = w0.flatten()
        return np.linalg.norm(w-w0)

def calc_normalized_error(w0, w, multi=False):
    w = np.array(w); w0 = np.array(w0)
    d = w.flatten().shape
    diff = w - w0
    err = np.abs(diff)
    err_total = np.sum(err)
    return err_total/d

def calc_rmse(y_,y):
    return np.sqrt(np.mean((y.flatten() - y_.flatten())**2))

def normalize(y):
    return (y-np.min(y))/(np.max(y) - np.min(y))


def shuffle_up_to_delta(y, eps, return_groups=False):
    vals = np.linspace(np.min(y),np.max(y),(np.max(y)-np.min(y))/eps+1)
    groups = np.copy(y)
    #print("Actual delta:",vals[1]-vals[0])
    y_ = y.copy()
    for i in range(len(vals)-1):
        idx = np.where(np.logical_and(y>=vals[i], y<=vals[i+1]))
        y_[idx] = np.random.permutation(y[idx])
        groups[idx] = i
    #plt.plot(y, y_,'.')
    if (return_groups):
        return y_, groups
    return y_

def enhanced_sort(x, w, groups):
    for i in np.unique(groups):
        idx = np.where(groups==i)[0]
        order = np.argsort(x.dot(w).flatten()[idx])
        x[idx] = x[idx][order]
    return x

def sorted_distance(y, y_):
    y = np.sort(y.flatten())
    y_ = np.sort(y_.flatten())
    return np.linalg.norm(y-y_)

def ols(x,y,alpha=0.01,fit_intercept=False):
    if alpha==0:
        lr =  LinearRegression(fit_intercept=fit_intercept)
    else:
        lr = Ridge(fit_intercept=fit_intercept, alpha=alpha)
    lr.fit(x, y)
    w = lr.coef_.T
    return w


def shuffle_within_num_groups_by_feature(features, labels, feature_index, n_clusters = 1, fraction_missorted=0, random_assignment=False):
    n,d = features.shape
    labels = labels.flatten()

    cluster_vector = np.repeat(range(n_clusters), n//n_clusters)
    padding = np.repeat([n_clusters-1],n%n_clusters)
    cluster_vector = np.concatenate((cluster_vector, padding))

    #Handle the missorts (they get randomly put into a bin ahead or behind)
    missort_idx = np.random.permutation(n)[:int(fraction_missorted*n)]
    missort = list()
    for i in missort_idx:
        if cluster_vector[i]==0:
            missort.append(1)
        elif cluster_vector[i]==n_clusters:
            missort.append(-1)
        else:
            if np.random.random()<0.5:
                missort.append(-1)
            else:
                missort.append(1)
    cluster_vector[missort_idx] += np.array(missort, dtype=int)

    #sort the features and labels
    order = np.argsort(features[:,feature_index])
    #if (random_assignment):
        #order = np.random.permutation(order)

    features = features[order]
    labels = labels[order]


    #randomize the label ordering
    for i in range(n_clusters):
        idx = np.where(cluster_vector==i)
        labels[idx] = np.random.permutation(labels[idx])
        features[idx] = np.random.permutation(features[idx])

    return features, labels, cluster_vector

def shuffle_within_num_groups(features, labels, n_clusters = 1, fraction_missorted=0, random_assignment=False):
    n,d = features.shape
    labels = labels.flatten()

    cluster_vector = np.repeat(range(n_clusters), n//n_clusters)
    padding = np.repeat([n_clusters-1],n%n_clusters)
    cluster_vector = np.concatenate((cluster_vector, padding))

    #Handle the missorts (they get randomly put into a bin ahead or behind)
    missort_idx = np.random.permutation(n)[:int(fraction_missorted*n)]
    missort = list()
    for i in missort_idx:
        if cluster_vector[i]==0:
            missort.append(1)
        elif cluster_vector[i]==n_clusters:
            missort.append(-1)
        else:
            if np.random.random()<0.5:
                missort.append(-1)
            else:
                missort.append(1)
    cluster_vector[missort_idx] += np.array(missort, dtype=int)


    #sort the features and labels
    order = np.argsort(labels)
    #if (random_assignment):
        #order = np.random.permutation(order)

    features = features[order]
    labels = labels[order]


    #randomize the label ordering
    for i in range(n_clusters):
        idx = np.where(cluster_vector==i)
        labels[idx] = np.random.permutation(labels[idx])
        features[idx] = np.random.permutation(features[idx])

    return features, labels, cluster_vector

def calc_angle(w0, w):
    w = np.array(w); w0 = np.array(w0)
    w = w.flatten()
    w0 = w0.flatten()
    return np.arccos(np.dot(w,w0)/(np.linalg.norm(w)*np.linalg.norm(w0)))


def semi_mean_ols_y(features,labels, groups, alpha=0.01):
    if alpha==0:
        lr =  LinearRegression(fit_intercept=False)
    else:
        lr = Ridge(fit_intercept=False, alpha=alpha)

    xs = list()
    ys = list()
    for i in np.unique(groups):
        idxs = np.where(groups==i)[0]
        y = np.mean(labels[idxs],axis=0)
        for idx in idxs:
            x = features[idx,:]
            xs.append(x)
            ys.append(y)
    xs = np.array(xs)
    ys = np.array(ys)
    #print(xs.shape, ys.shape)
    lr.fit(xs, ys)
    return lr.coef_.T

def semi_mean_ols_x(features,labels, groups, alpha=0.01):
    if alpha==0:
        lr =  LinearRegression(fit_intercept=False)
    else:
        lr = Ridge(fit_intercept=False, alpha=alpha)
    xs = list()
    ys = list()
    for i in np.unique(groups):
        idxs = np.where(groups==i)[0]
        x = np.mean(features[idxs,:],axis=0)
        for idx in idxs:
            xs.append(x)
            y = labels[idx]
            ys.append(y)
    xs = np.array(xs)
    ys = np.array(ys)
    #print(xs.shape, ys.shape)
    lr.fit(xs, ys)
    return lr.coef_.T

def mean_ols(features,labels, groups, alpha=0.01):
    if alpha==0:
        lr =  LinearRegression(fit_intercept=False)
    else:
        lr = Ridge(fit_intercept=False, alpha=alpha)
    xs = list()
    ys = list()
    for i in np.unique(groups):
        idxs = np.where(groups==i)[0]
        x = np.mean(features[idxs,:],axis=0)
        y = np.mean(labels[idxs],axis=0)
        for idx in idxs:
            xs.append(x)
            ys.append(y)
    xs = np.array(xs)
    ys = np.array(ys)
    lr.fit(xs, ys)
    return lr.coef_.T

def mean_ols_with_noise(features,labels, groups, noise=0.01, alpha=0.01):
    if alpha==0:
        lr =  LinearRegression(fit_intercept=False)
    else:
        lr = Ridge(fit_intercept=False, alpha=alpha)
    xs = list()
    ys = list()
    for i in np.unique(groups):
        for _ in range(100):
            idx = np.where(groups==i)[0]
            x = np.mean(features[idx,:],axis=0)
            xs.append(x)
            y = np.mean(labels[idx],axis=0)
            y = y + np.random.normal(0,noise,size=y.shape)
            ys.append(y)
    xs = np.array(xs)
    ys = np.array(ys)
    #print(xs.shape, ys.shape)
    lr.fit(xs, ys)
    return lr.coef_.T

def generate_distribution(dim=2, n=100, noise=0, dist='normal', mean=0, var=1, WA=None, bias=False, n_clusters=None):
    """generates data of a given dimension and distribution with given parameters
    WA -- if you would like to set the weight matrix, provide it here
    """
    if (dist=='normal'):
        X = np.random.normal(mean,var, size=[n, dim]);
    elif (dist=='half-normal-uniform'):
        X1 = np.random.rand(n, dim//2)-mean*0.5
        X2 = np.random.normal(mean, var, size=[n, dim//2])
        X = np.concatenate((X1, X2), axis=1)
    elif (dist=='uniform'):
        X = var*np.random.rand(n, dim)+mean
    elif (dist=='2normals'):
        X1 = np.random.normal(-mean,var, size=[n/2, dim])
        X2 = np.random.normal(mean,var, size=[n-n/2, dim]);
        X = np.concatenate((X1, X2), axis=0)
    elif (dist=='exponential'):
        X = np.random.exponential(var, size=[n, dim]);
    else:
        raise NameError('Invalid distribution: ' + str(dist))
    if (bias):
        X[:,0] = 1 #set the first column to be all 1s for the "intercept" term
    if (WA is None):
        WA = np.random.normal(0,1,size=[dim, 1]); #Actual weights
    else:
        WA = np.array(WA)
        WA = WA.reshape(dim, 1)
    y = np.dot(X,WA) + noise*np.random.normal(0,1,size=[n,1]); #Ordered labels
    if (n_clusters is None):
        return X, y, WA
    else:
        cluster_vector = np.tile(range(n_clusters), n//n_clusters)
        cluster_vector = np.concatenate((cluster_vector, range(n%n_clusters)))
        np.random.shuffle(cluster_vector)
        return X, y, WA, cluster_vector

def load_dataset_clusters(filename, normalize=True, n_clusters = 1):
    data = np.genfromtxt(filename, delimiter=',',skip_header=2)
    n,d = data.shape

    cluster_vector = np.tile(range(n_clusters), n//n_clusters)
    cluster_vector = np.concatenate((cluster_vector, range(n%n_clusters)))
    np.random.shuffle(cluster_vector)

    #normalize
    if (normalize):
        data = (data-np.min(data, axis=0))
        data = data/(np.max(data, axis=0)+0.01)

    labels = data[:,-1].copy()
    features = data[:,:].copy()
    features[:,-1] = 1 #add a bias column

    return features, labels, cluster_vector


In [7]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from scipy.stats import norm
from utils import *

# Stochastic EM Algorithm
def em_mcmc(x,y,fit_intercept=False,steps=15,return_all_weights=False,verbose=False, enhanced=False, groups=None, mcmc_steps=None, interval_between_mcmc_steps=None):
    import time

    lr =  LinearRegression(fit_intercept=fit_intercept)
    n, d = x.shape
    lr.fit(x,y)
    coefs = lr.coef_.T
    s = np.sqrt(np.sum(np.square(y - x.dot(coefs)))/(n-d)) #RMS of residuals

    ws = list()

    for i in list(range(steps)):

        order = np.arange(n)
        y_mean = np.zeros(n)

        if mcmc_steps is None:
            mcmc_steps = np.max([int(n*np.log(n)),100])
        if interval_between_mcmc_steps is None:
            interval_between_mcmc_steps = n/10
        burn_steps = int(mcmc_steps/2)
        mean_step_counter = 0
        interval_mcmc_counter = 0

        for m_step in range(mcmc_steps):
            order_ = np.copy(order)

            i = np.random.randint(0,n)

            if (enhanced):
                if not(groups is None):
                    indices_in_same_group = np.where(groups==groups[i])[0]
                    j = np.random.choice(indices_in_same_group)
                else:
                    raise ValueError("Group Information Must Be Provided for Enhanced SLS")
            else:
                j = np.random.randint(0,n)

            order_[i] = order[j]
            order_[j] = order[i]

            p1 = log_prob_of_order(x,y,coefs,s,order)
            p2 = log_prob_of_order(x,y,coefs,s,order_)

            if p1 < p2:
                order = order_
            else:
                if np.random.random() < np.exp(p2-p1): #since they're log probabilities
                    order = order_

            if m_step>=burn_steps:
                interval_mcmc_counter += 1
                if (interval_mcmc_counter>=interval_between_mcmc_steps):
                    y_mean += y.flatten()[order]
                    mean_step_counter += 1
                    interval_mcmc_counter = 0

        y = y_mean/mean_step_counter
        y = y.reshape(-1,1)

        lr.fit(x,y)
        coefs = lr.coef_.T
        s = np.sqrt(np.sum(np.square(y - x.dot(coefs)))/(n-d)) #RMS of residuals
        ws.append(coefs)
    residual=np.linalg.norm(y - x.dot(coefs))/np.linalg.norm(y)
    
    if (return_all_weights):
        return ws
    return coefs,residual

# Hard EM Algorithm
def sls(x, y, steps=15, alpha=0, w_initial=None, return_all_weights = False, enhanced=False, groups=None, fit_intercept=False, n_starts=1):
    if alpha==0:
        lr =  LinearRegression(fit_intercept=fit_intercept)
    else:
        lr = Ridge(fit_intercept=fit_intercept, alpha=alpha)

    order = np.argsort(y.flatten())
    y = y[order]
    x = x[order]
    errors = list()
    optimal_score = float("-inf")
    optimal_ws = list()

    for i_start in range(n_starts):
        ws = list()
        if w_initial is None and n_starts==1:
            lr.fit(x, y)
            w = lr.coef_.T
        elif w_initial is None:
            if i_start>0:
                y_ = np.random.permutation(y)
            elif i_start==0: #try the current permutation first
                y_ = y
            lr.fit(x, y_)
            w = lr.coef_.T
        elif w_initial=='mean':
            y_ = np.copy(y)
            for i in np.unique(groups):
                idx = np.where(groups==i)[0]
                y_[idx] = np.mean(y[idx])
            lr.fit(x, y_)
            w = lr.coef_.T
        else:
            w = w_initial

        for _ in list(range(steps)):

            if (enhanced):
                if not(groups is None):
                    x = enhanced_sort(x, w, groups)
                else:
                    raise ValueError("Group Information Must Be Provided for Enhanced SLS")
            else:
                order = np.argsort(x.dot(w).flatten())
                x = x[order]

            lr.fit(x, y)
            w = lr.coef_.T
            ws.append(w)

        if lr.score(x,y)>optimal_score:
            optimal_score = lr.score(x,y)
            optimal_ws = ws.copy()

    if (return_all_weights):
        return optimal_ws
    return optimal_ws[-1]

In [8]:
import sys, os;
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
#from utils import *
#from algorithms import *

%load_ext autoreload
%matplotlib inline
%autoreload 2

np.set_printoptions(precision=2)
lr =  LinearRegression(fit_intercept=False)

font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 18}

matplotlib.rc('font', **font)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
def generatedata(noise,showpermutation=False,showtrue_w=False):
    true_w2 = np.random.normal(0, 1,(num_X2feature,1))
    X2_before_ =np.random.normal(0, 1, (num_example, num_X2feature))
    X2_test_before=np.random.normal(0, 1, (num_example, num_X2feature))
    y_ = np.matmul(X2_before_,true_w2)
    y_test=np.matmul(X2_test_before,true_w2)
    y_ += np.random.normal(0, noise ,size=(num_example,1))
    y_test += np.random.normal(0, noise ,size=(num_example,1))
    P_array=np.random.permutation(num_example)
    P=np.zeros((num_example,num_example))
    for i in range(num_example):
        P[i][P_array[i]]=1
    if showpermutation:
        print('打乱X2的置换矩阵为',P)
    P_arraytest=np.random.permutation(num_example)
    P_test=np.zeros((num_example,num_example))
    for i in range(num_example):
        P_test[i][P_arraytest[i]]=1
    X2_=np.matmul(P,X2_before_)
    X2_test=np.matmul(P_test,X2_test_before)
    conditionnumber=np.linalg.cond(X2_)
    #X2_=X2_before_
    error_reg=(np.linalg.norm(y_-np.matmul(X2_before_,true_w2))/np.linalg.norm(y_))
    error_reg_test=(np.linalg.norm(y_test-np.matmul(X2_test_before,true_w2))/np.linalg.norm(y_test))
    return y_,X2_,true_w2,P,error_reg,conditionnumber,y_test,X2_test,P_test,error_reg_test

In [10]:
import sys, os;
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
#from utils import *
#from algorithms import *

%load_ext autoreload
%matplotlib inline
%autoreload 2

np.set_printoptions(precision=2)
lr =  LinearRegression(fit_intercept=False)

font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 18}

matplotlib.rc('font', **font)
d = 30
noise = 1
iters = 5
ns = np.linspace(100,500,5,dtype=int)

error_hard = np.zeros((iters,len(ns)))
error_soft = np.zeros((iters,len(ns)))

for i in range(iters):
    print(i, end='| ')
    x_, y__, w0_ = generate_distribution(n=np.max(ns), dim=d,  dist='normal', bias=False, noise=0)
    y_ = y__ + np.random.normal(0,noise,y__.shape)
    for n_i, n in enumerate(ns):
        #print(n, end=' ')
        y = y_[:n]; x = x_[:n]
        y = np.random.permutation(y)
        weights,residual = em_mcmc(x,y,steps=50,return_all_weights=False)
        error = calc_error(w0_, weights)
        error1=np.linalg.norm(weights-w0_)/np.linalg.norm(w0_)
        print(n,'w相对误差',error1,'实验回归残差',residual)
        error_soft[i,n_i] = error

        #weights = sls(x,y,steps=50,return_all_weights=True, n_starts=n)
        #error = calc_error(w0_, weights[-1])
        #error_hard[i,n_i] = error

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
0| 100 w相对误差 1.021444711137078 实验回归残差 0.358686249933205
200 w相对误差 1.0403542034560562 实验回归残差 0.9079873409249418
300 w相对误差 0.9987857849744898 实验回归残差 0.8561369989089445
400 w相对误差 1.0299783585370452 实验回归残差 0.8443449332962878
500 w相对误差 1.0069713813111547 实验回归残差 0.9512080763610316
1| 100 w相对误差 1.0999613105379245 实验回归残差 0.35840551423508954
200 w相对误差 0.9976921826961848 实验回归残差 0.7789241137476555
300 w相对误差 0.9609557938326696 实验回归残差 0.8814305669767049
400 w相对误差 0.9818042607481612 实验回归残差 0.8996292960872451
500 w相对误差 0.9969350469401662 实验回归残差 0.9079419423098971
2| 100 w相对误差 1.049945863807408 实验回归残差 0.6415475260949608
200 w相对误差 0.9836203475395292 实验回归残差 0.8534880525765105
300 w相对误差 0.9595798005398224 实验回归残差 0.832548090976611
400 w相对误差 0.9829541409921443 实验回归残差 0.8879957979334031
500 w相对误差 0.991604234751429 实验回归残差 0.927198590692457
3| 100 w相对误差 1.2077684827720025 实验回归残差 0.33310834357994834
200 w相对误差 1.001116874135